[使用sklearn之LabelEncoder将Label标准化](https://blog.csdn.net/u010412858/article/details/78386407)

[【Python 课堂】第65课—— pickle](http://www.kokojia.com/article/3645.html)

[NumPy 1.14 教學 – #01 基礎, 建立陣列的方法](https://www.brilliantcode.net/1022/numpy-tutorial-basics-array-creations/)

[10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html)

[def 陳述句](https://openhome.cc/Gossip/Python/DefStatement.html)

In [1]:
import os
# import face_recognition_api
import face_recognition as face_recognition_api
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

In [2]:
def _get_training_dirs(training_dir_path):
    return [x[0] for x in os.walk(training_dir_path)][1:]


def _get_training_labels(training_dir_path):
    return [x[1] for x in os.walk(training_dir_path)][0]


def _get_each_labels_files(training_dir_path):
    return [x[2] for x in os.walk(training_dir_path)][1:]


def _filter_image_files(training_dir_path):
    exts = [".jpg", ".jpeg", ".png"]

    training_folder_files_list = []
    for list_files in _get_each_labels_files(training_dir_path):
        l = []
        for file in list_files:
            imageName, ext = os.path.splitext(file)
            if ext.lower() in exts:
                l.append(file)
        training_folder_files_list.append(l)

    return training_folder_files_list


def _zipped_folders_labels_images(training_dir_path, labels):
    return list(zip(_get_training_dirs(training_dir_path),
                    labels,
                    _filter_image_files(training_dir_path)))


def create_dataset(training_dir_path, labels):
    X = []
    for i in _zipped_folders_labels_images(training_dir_path, labels):
        for fileName in i[2]:
            file_path = os.path.join(i[0], fileName)
            img = face_recognition_api.load_image_file(file_path)
#             X_faces_loc = face_recognition_api.face_locations(img, model="cnn")
            X_faces_loc = face_recognition_api.face_locations(img)
            imgEncoding = face_recognition_api.face_encodings(img, known_face_locations=X_faces_loc)

            if len(imgEncoding) > 1:
                print('\x1b[0;37;43m' + 'More than one face found in {}. Only considering the first face.'.format(file_path) + '\x1b[0m')
            if len(imgEncoding) == 0:
                print('\x1b[0;37;41m' + 'No face found in {}. Ignoring file.'.format(file_path) + '\x1b[0m')
            else:
                print('Encoded {} successfully.'.format(file_path))
                X.append(np.append(imgEncoding[0], i[1]))
    return X

In [4]:
encoding_file_path = './encoded-images-data.csv'
training_dir_path = './training-images'
labels_fName = "labels.pkl"

In [5]:
# os.walk('./training-images')
# [('當前目錄' , [下一層目錄] , [當層檔案] )]
'''
[('./training-images', ['胡瓜 Hu gua', '謝忻 Xie Xin', '阿翔 A Xiang'], ['004.JPG']),
 ('./training-images\\胡瓜 Hu gua', [], ['005 (2).JPG', '擷取.JPG']),
 ('./training-images\\謝忻 Xie Xin', [], ['003.JPG', '擷3取.JPG']),
 ('./training-images\\阿翔 A Xiang',
  ['testt'],
  ['00004.JPG',
   '001.JPG',
   '003.JPG',
   '004.JPG',
   '009.JPG',
   '010.JPG',
   '擷取9.JPG']),
 ('./training-images\\阿翔 A Xiang\\testt', [], [])]
'''

print([x[0] for x in os.walk(training_dir_path)][1:])
print([x[1] for x in os.walk(training_dir_path)][0])
print([x[2] for x in os.walk(training_dir_path)][1:])

['./training-images\\胡瓜 Hu gua', './training-images\\謝忻 Xie Xin', './training-images\\阿翔 A Xiang', './training-images\\阿翔 A Xiang\\testt']
['胡瓜 Hu gua', '謝忻 Xie Xin', '阿翔 A Xiang']
[['005 (2).JPG', '擷取.JPG'], ['003.JPG', '3faces.JPG', '擷3取.JPG'], ['00004.JPG', '001.JPG', '003.JPG', '004.JPG', '009.JPG', '010.JPG', '擷取9.JPG'], []]


In [6]:
# os.walk('./training-images')

In [7]:
[i for i in _zipped_folders_labels_images(training_dir_path, labels)]

NameError: name 'labels' is not defined

In [8]:
# Get the folder names in training-dir as labels
# Encode them in numerical form for machine learning
labels = _get_training_labels(training_dir_path)
print(labels)
le = LabelEncoder().fit(labels)
labelsNum = le.transform(labels)
print(labelsNum)
nClasses = len(le.classes_)
print(nClasses)
dataset = create_dataset(training_dir_path, labelsNum)
df = pd.DataFrame(dataset)
len(df)

['胡瓜 Hu gua', '謝忻 Xie Xin', '阿翔 A Xiang']
[0 1 2]
3
Encoded ./training-images\胡瓜 Hu gua\005 (2).JPG successfully.
Encoded ./training-images\胡瓜 Hu gua\擷取.JPG successfully.
Encoded ./training-images\謝忻 Xie Xin\003.JPG successfully.
More than one face found in ./training-images\謝忻 Xie Xin\3faces.JPG. Only considering the first face.
Encoded ./training-images\謝忻 Xie Xin\3faces.JPG successfully.
Encoded ./training-images\謝忻 Xie Xin\擷3取.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\00004.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\001.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\003.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\004.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\009.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\010.JPG successfully.
Encoded ./training-images\阿翔 A Xiang\擷取9.JPG successfully.


12

In [70]:
df

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,-0.027006,0.057856,0.025634,-0.058075,-0.030775,-0.054881,-0.036883,-0.110501,0.072271,-0.116070,...,0.172654,0.010867,0.058486,-0.237354,-0.034140,0.115628,0.020436,0.079115,-0.023658,0.0
1,-0.049976,0.054397,-0.011805,-0.029721,-0.060572,-0.036806,-0.011119,-0.103832,0.059076,-0.062082,...,0.144966,0.027073,-0.009467,-0.249590,-0.019263,0.164780,0.039644,0.098230,0.021896,0.0
2,-0.096459,0.065650,0.093981,-0.099006,-0.183421,-0.080738,-0.052799,-0.087779,0.113442,-0.157002,...,0.040972,-0.004417,0.027891,-0.205696,-0.033126,0.109614,0.010493,0.043472,0.012393,1.0
3,-0.116502,0.141658,0.115101,-0.115511,-0.140740,-0.066602,-0.121032,-0.114812,0.160056,-0.072735,...,0.040734,0.009015,-0.042008,-0.214065,0.002871,0.068806,-0.086796,0.142464,0.048777,1.0
4,-0.074300,0.050935,0.091982,-0.102513,-0.178154,-0.077791,-0.042596,-0.088722,0.117715,-0.175710,...,0.049680,-0.028611,0.044787,-0.210466,-0.030242,0.103928,0.004133,0.040149,0.000182,1.0
5,-0.121135,0.078943,0.022784,-0.046896,-0.171911,0.013131,-0.058301,-0.092381,0.125721,-0.055963,...,0.023688,-0.004369,-0.044434,-0.198595,-0.037817,0.087412,-0.055752,0.151675,0.033223,2.0
6,-0.078656,0.051062,0.041625,-0.038913,-0.125728,0.024614,-0.086170,-0.130084,0.105704,-0.132270,...,0.021555,0.005318,-0.035353,-0.233997,0.008135,0.123798,-0.019486,0.129150,-0.022151,2.0
7,-0.083741,0.068369,0.012402,-0.022855,-0.144247,-0.025872,-0.071074,-0.085706,0.121345,-0.074792,...,0.034842,-0.014642,-0.055266,-0.229068,-0.033602,0.087797,-0.010127,0.096185,0.001098,2.0
8,-0.133337,0.087505,0.028327,-0.031949,-0.148272,0.026420,-0.092480,-0.122086,0.090636,-0.072743,...,0.053198,-0.009721,-0.017743,-0.234638,-0.022253,0.114749,-0.046502,0.113058,0.032383,2.0
9,-0.105492,0.054206,0.034959,-0.009625,-0.143223,-0.011947,0.002192,-0.134863,0.085794,-0.113329,...,-0.025448,0.001099,-0.047367,-0.263002,-0.064010,0.162366,-0.040370,0.175796,-0.018311,2.0


In [9]:
# if file with same name already exists, backup the old file
if os.path.isfile(encoding_file_path):
    print("{} already exists. Backing up.".format(encoding_file_path))
    try:
        os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))
    except WindowsError:
        os.remove("{}.bak".format(encoding_file_path))
        os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))
#    os.rename(encoding_file_path, "{}.bak".format(encoding_file_path))

df.to_csv(encoding_file_path)

print("{} classes created.".format(nClasses))
print('\x1b[6;30;42m' + "Saving labels pickle to'{}'".format(labels_fName) + '\x1b[0m')
with open(labels_fName, 'wb') as f:
    pickle.dump(le, f)
print('\x1b[6;30;42m' + "Training Image's encodings saved in {}".format(encoding_file_path) + '\x1b[0m')

./encoded-images-data.csv already exists. Backing up.
3 classes created.
Saving labels pickle to'labels.pkl'
Training Image's encodings saved in ./encoded-images-data.csv


In [66]:
import time
#####################
### create_encodings
stime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",stime))
for i in range(0,100000000,1):
    1+1
etime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",etime))
#####################

2019-02-20 11:14:21
2019-02-20 11:14:24


In [67]:
print( time.localtime( time.time() ) )

time.struct_time(tm_year=2019, tm_mon=2, tm_mday=20, tm_hour=11, tm_min=14, tm_sec=24, tm_wday=2, tm_yday=51, tm_isdst=0)


In [68]:
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )

2019-02-20 11:14:24
